In [1]:
import datetime
from os import rename
from os.path import splitext
from calendar import month_name
from httplib import HTTPConnection
from time import clock
from urllib import urlretrieve
import urllib2
from zipfile import ZipFile
import numpy as np
from bs4 import BeautifulSoup

from data_scraper import *

import os
import errno
import humanize
import sys
import json
import pandas as pd

In [84]:
def parseHistoryTable(table):
    temperature = 0 # in degree F
    events = '' # some string i.e. rain
    humidity = 0 # humidity in %
    precipitation = 0 # in inch
    sealevelpressure = 0 # in inch
    visibility = 0 # in miles
    windspeed = 0 # in miles per hour

    # skip the first entries of some fields...
    skipprec = True
    skippres = True

    # Find all the <tr> tag pairs, skip the first one, then for each.
    for row in table.find_all('tr')[1:]:
        # Create a variable of all the <td> tag pairs in each <tr> tag pair,
        col = row.find_all('td')

        # only retrieve relevant columns
        if col[0].text == 'Mean Temperature':
            temperature = int(col[1].find('span', attrs={"class": "wx-value"}).text)
        if col[0].text == 'Average Humidity':
            humidity = int(col[1].text)
        if col[0].text == 'Precipitation':
            if not skipprec:
                precipitation = float(col[1].find('span', attrs={"class": "wx-value"}).text)
            skipprec = False
        if col[0].text == 'Sea Level Pressure':
            if not skippres:
                sealevelpressure = float(col[1].find('span', attrs={"class": "wx-value"}).text)
            skippres = False
        if col[0].text == 'Visibility':
            visibility = float(col[1].find('span', attrs={"class": "wx-value"}).text)
        if col[0].text == 'Wind Speed':
            windspeed = float(col[1].find('span', attrs={"class": "wx-value"}).text)
        if col[0].text == 'Events':
            events = str(col[1].text).strip()
    # return as dictionary
    d = dict(zip(['temperature', 'events', 'humidity', 'precipitation', 'sealevelpressure', 'visibility', 'windspeed'], \
    [temperature, events, humidity, precipitation, sealevelpressure, visibility, windspeed]))
    
    return d

def getWeather(year, day, month, airportcode):
    url = 'http://www.wunderground.com/cgi-bin/findweather/getForecast?airportorwmo=query&historytype=DailyHistory&backurl=%2Fhistory%2Findex.html&code={airportcode}&month={month}&day={day}&year={year}'
    response = urllib2.urlopen(url.format(year=year, day=day, month=month, airportcode=airportcode))
    html = response.read()
    
    soup = BeautifulSoup(html, "html.parser")
    
    table = soup.find("table", attrs={"id": "historyTable"})
    
    return parseHistoryTable(table)

In [85]:
year = 2015
day = 3
month = 10
airportcode = 'FRA'

getWeather(2015, 3, 10, airportcode)

{'events': 'Fog',
 'humidity': 66,
 'precipitation': 0.0,
 'sealevelpressure': 30.0,
 'temperature': 57,
 'visibility': 15.8,
 'windspeed': 6.0}

In [99]:
# lazy load dictionary
weatherDict = {}
weatherFile = os.path.join('..', 'cache', 'weather_data.json')
if file_exists(weatherFile):
    # load current dict from json to cache results
    with open(weatherFile) as infile:
        weatherDict = json.load(infile)
    



In [100]:
file_exists(weatherFile)

False